# 1. Import Libaries

## 1A) Import Libaries

In [1]:
# Import the necessary libraries

# gymnasium is a modern version of the gym library used to create and interact with reinforcement learning environments
import gymnasium as gym

# Import PPO (Proximal Policy Optimization) from stable-baselines3, which is a popular RL algorithm
from stable_baselines3 import PPO

# Import the evaluation function to assess the performance of the trained policy
from stable_baselines3.common.evaluation import evaluate_policy

## 1B) Create Env and Test

In [ ]:
# Create the BipedalWalker environment with human-rendering mode enabled
env = gym.make("BipedalWalker-v3", render_mode="human")

In [ ]:
# Reset the environment (start a new episode) - without using seed or options
obs = env.reset()

# Let the agent take random actions for 1000 steps
for _ in range(1000):
    # Take a random action sampled from the environment's action space
    action = env.action_space.sample()
    
    # Step the environment forward using the chosen action
    # The environment returns the new observation (obs), the reward, 
    # whether the episode is done (done), if it was truncated (truncated), and additional info (info)
    obs, reward, done, truncated, info = env.step(action)
    
    # If the episode is finished (either done or truncated), reset the environment for a new episode
    if done or truncated:
        obs = env.reset()

# Close the environment when finished to clean up resources
env.close()

# 2) Train Model for Normal Version with PPO

## 2A) Train Model

In [ ]:
env = gym.make("BipedalWalker-v3")

In [ ]:
# Create the PPO model with a Multi-Layer Perceptron (MLP) policy
model = PPO("MlpPolicy", env, verbose=1)

In [ ]:
model.learn(total_timesteps=1000000)

## 2B) Save Model

In [ ]:
model.save("ppo_bipedalwalker_1M")

In [ ]:
del model

## 2C) Evaluate Model

In [ ]:
model = PPO.load("ppo_bipedalwalker")

In [ ]:
env = gym.make("BipedalWalker-v3", render_mode="human")

In [ ]:
# Modeli değerlendirin (örneğin, 10 bölüm boyunca)
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)

print(f"Ortalama ödül: {mean_reward} ± {std_reward}")

# 3) Train Model for Hardcore Version with PPO

## 3A) Test Enviroment

In [ ]:
env = gym.make("BipedalWalker-v3", hardcore=True, render_mode="human")

In [ ]:
# Reset the environment (start a new episode) - without using seed or options
obs = env.reset()

# Let the agent take random actions for 1000 steps
for _ in range(1000):
    # Take a random action sampled from the environment's action space
    action = env.action_space.sample()
    
    # Step the environment forward using the chosen action
    # The environment returns the new observation (obs), the reward, 
    # whether the episode is done (done), if it was truncated (truncated), and additional info (info)
    obs, reward, done, truncated, info = env.step(action)
    
    # If the episode is finished (either done or truncated), reset the environment for a new episode
    if done or truncated:
        obs = env.reset()

# Close the environment when finished to clean up resources
env.close()

## 3B) Train Model

In [3]:
# Create the PPO model with a Multi-Layer Perceptron (MLP) policy
model = PPO("MlpPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [2]:
env = gym.make("BipedalWalker-v3", hardcore=True)

In [5]:
model.learn(total_timesteps=2000000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 362      |
|    ep_rew_mean     | -76.1    |
| time/              |          |
|    fps             | 4365     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 981         |
|    ep_rew_mean          | -64.1       |
| time/                   |             |
|    fps                  | 3558        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.023978543 |
|    clip_fraction        | 0.234       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.37        |
|    explained_variance   | 0.836       |
|    learning_rate        | 0.

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 973         |
|    ep_rew_mean          | -65.4       |
| time/                   |             |
|    fps                  | 3110        |
|    iterations           | 11          |
|    time_elapsed         | 7           |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.016823713 |
|    clip_fraction        | 0.169       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.389       |
|    explained_variance   | 0.289       |
|    learning_rate        | 0.0003      |
|    loss                 | 3.88        |
|    n_updates            | 4990        |
|    policy_gradient_loss | -0.0131     |
|    std                  | 0.236       |
|    value_loss           | 9.75        |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.01e+03    |
|    ep_rew_mean          | -65.1       |
| time/                   |             |
|    fps                  | 3056        |
|    iterations           | 20          |
|    time_elapsed         | 13          |
|    total_timesteps      | 40960       |
| train/                  |             |
|    approx_kl            | 0.022074915 |
|    clip_fraction        | 0.248       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.455       |
|    explained_variance   | 0.951       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.317       |
|    n_updates            | 5080        |
|    policy_gradient_loss | -0.0125     |
|    std                  | 0.233       |
|    value_loss           | 1.7         |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.05e+03    |
|    ep_rew_mean          | -63.7       |
| time/                   |             |
|    fps                  | 3024        |
|    iterations           | 29          |
|    time_elapsed         | 19          |
|    total_timesteps      | 59392       |
| train/                  |             |
|    approx_kl            | 0.015401494 |
|    clip_fraction        | 0.183       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.569       |
|    explained_variance   | 0.706       |
|    learning_rate        | 0.0003      |
|    loss                 | 3.62        |
|    n_updates            | 5170        |
|    policy_gradient_loss | -0.00748    |
|    std                  | 0.23        |
|    value_loss           | 10.4        |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 978         |
|    ep_rew_mean          | -61.3       |
| time/                   |             |
|    fps                  | 3010        |
|    iterations           | 38          |
|    time_elapsed         | 25          |
|    total_timesteps      | 77824       |
| train/                  |             |
|    approx_kl            | 0.015435314 |
|    clip_fraction        | 0.148       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.573       |
|    explained_variance   | 0.918       |
|    learning_rate        | 0.0003      |
|    loss                 | 6.48        |
|    n_updates            | 5260        |
|    policy_gradient_loss | -0.0127     |
|    std                  | 0.231       |
|    value_loss           | 18          |
-----------------------------------------
-----------------------------------------
| rollout/                |       

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 752         |
|    ep_rew_mean          | -57.6       |
| time/                   |             |
|    fps                  | 2771        |
|    iterations           | 934         |
|    time_elapsed         | 690         |
|    total_timesteps      | 1912832     |
| train/                  |             |
|    approx_kl            | 0.021611463 |
|    clip_fraction        | 0.258       |
|    clip_range           | 0.2         |
|    entropy_loss         | 3.64        |
|    explained_variance   | 0.868       |
|    learning_rate        | 0.0003      |
|    loss                 | 14.5        |
|    n_updates            | 14220       |
|    policy_gradient_loss | -0.0175     |
|    std                  | 0.102       |
|    value_loss           | 40.3        |
-----------------------------------------
----------------------------------------
| rollout/                |        

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 777         |
|    ep_rew_mean          | -56.1       |
| time/                   |             |
|    fps                  | 2770        |
|    iterations           | 943         |
|    time_elapsed         | 697         |
|    total_timesteps      | 1931264     |
| train/                  |             |
|    approx_kl            | 0.077048674 |
|    clip_fraction        | 0.427       |
|    clip_range           | 0.2         |
|    entropy_loss         | 3.54        |
|    explained_variance   | 0.463       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.148       |
|    n_updates            | 14310       |
|    policy_gradient_loss | 0.00017     |
|    std                  | 0.104       |
|    value_loss           | 0.732       |
-----------------------------------------
----------------------------------------
| rollout/                |        

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 745        |
|    ep_rew_mean          | -60.1      |
| time/                   |            |
|    fps                  | 2769       |
|    iterations           | 952        |
|    time_elapsed         | 703        |
|    total_timesteps      | 1949696    |
| train/                  |            |
|    approx_kl            | 0.05227442 |
|    clip_fraction        | 0.405      |
|    clip_range           | 0.2        |
|    entropy_loss         | 3.62       |
|    explained_variance   | -0.262     |
|    learning_rate        | 0.0003     |
|    loss                 | 0.0267     |
|    n_updates            | 14400      |
|    policy_gradient_loss | -0.00518   |
|    std                  | 0.103      |
|    value_loss           | 0.381      |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 698        |
|    ep_rew_mean          | -61.5      |
| time/                   |            |
|    fps                  | 2769       |
|    iterations           | 961        |
|    time_elapsed         | 710        |
|    total_timesteps      | 1968128    |
| train/                  |            |
|    approx_kl            | 0.04778247 |
|    clip_fraction        | 0.318      |
|    clip_range           | 0.2        |
|    entropy_loss         | 3.61       |
|    explained_variance   | 0.951      |
|    learning_rate        | 0.0003     |
|    loss                 | 1.4        |
|    n_updates            | 14490      |
|    policy_gradient_loss | -0.0109    |
|    std                  | 0.102      |
|    value_loss           | 5.13       |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_me

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 763        |
|    ep_rew_mean          | -59        |
| time/                   |            |
|    fps                  | 2768       |
|    iterations           | 970        |
|    time_elapsed         | 717        |
|    total_timesteps      | 1986560    |
| train/                  |            |
|    approx_kl            | 0.05372051 |
|    clip_fraction        | 0.367      |
|    clip_range           | 0.2        |
|    entropy_loss         | 3.59       |
|    explained_variance   | -0.57      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.527      |
|    n_updates            | 14580      |
|    policy_gradient_loss | -0.00478   |
|    std                  | 0.104      |
|    value_loss           | 1.07       |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_me

## 3C) Save Model

In [6]:
model.save("ppo_bipedalwalker_hardcore_3M")

## 3D) Evaluate Model

In [7]:
env = gym.make("BipedalWalker-v3", hardcore=True, render_mode="human")

In [8]:
# Modeli değerlendirin (örneğin, 10 bölüm boyunca)
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)

print(f"Ortalama ödül: {mean_reward} ± {std_reward}")

/Users/can/anaconda3/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
2024-09-24 15:39:31.035 python[5914:203471] +[IMKClient subclass]: chose IMKClient_Legacy
2024-09-24 15:39:31.035 python[5914:203471] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Ortalama ödül: -53.9464984558479 ± 34.84904356642908
